<a href="https://colab.research.google.com/github/AxelCodaeMolina/Parids-calls/blob/main/map.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json
import pandas as pd
from shapely.geometry import Point
from itertools import combinations
from shapely.ops import unary_union
import geopandas as gpd
import numpy as np

# Function to read GeoJSON data from a file
def read_geojson(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return json.load(file)

# Read the GeoJSON data from the file
file_path = '/content/CTMap.geojson'
geojson_data = read_geojson(file_path)

# Extract coordinates from GeoJSON
points = []
for feature in geojson_data['features']:
    point_coords = feature['geometry']['coordinates']
    points.append({'name': feature['properties']['name'], 'coordinates': point_coords})

# Create a DataFrame with coordinates
df = pd.DataFrame(points)
df['geometry'] = df['coordinates'].apply(Point)

# Convert to a GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry='geometry', crs='EPSG:4326')

# Reproject to a local projection
gdf = gdf.to_crs('EPSG:32631')

# Function to calculate pairwise distances in meters and filter <= 500 meters
def calculate_pairwise_distances(gdf):
    distances = []
    for i, j in combinations(gdf.index, 2):
        distance = gdf.loc[i, 'geometry'].distance(gdf.loc[j, 'geometry'])
        if distance <= 500:  # Filter distances <= 500 meters
            distances.append({
                'point1': gdf.loc[i, 'name'],
                'point2': gdf.loc[j, 'name'],
                'distance_meters': distance
            })
    return pd.DataFrame(distances)

# Calculate pairwise distances and filter <= 500 meters
distances_df = calculate_pairwise_distances(gdf)

# Find the minimum distance for each point
min_distances = distances_df.groupby('point1')['distance_meters'].min()

# Calculate the average minimum distance in meters
avg_min_distance_meters = min_distances.mean()

# Calculate the standard deviation (SD) of the minimum distances
std_min_distance = min_distances.std()

# Calculate the standard error (SE) of the minimum distances
se_min_distance = std_min_distance / np.sqrt(len(min_distances))

# Output the closest distances for verification
closest_distances = distances_df.sort_values(by='distance_meters').head()
print("Closest distances in meters:")
print(closest_distances)

# Output the average minimum distance in meters
print(f"\nAverage minimum distance between points in meters: {avg_min_distance_meters} meters")

# Output the standard deviation of the minimum distances
print(f"Standard deviation of the minimum distances: {std_min_distance} meters")

# Output the standard error of the minimum distances
print(f"Standard error of the minimum distances: {se_min_distance} meters")

# Calculate the convex hull of the points
convex_hull = gdf.unary_union.convex_hull

# Calculate the area of the convex hull
area = convex_hull.area

# Output the area
print(f"The area covered by the points is: {area} square meters")


Closest distances in meters:
            point1        point2  distance_meters
1360  Marqueur 529  Marqueur 530        44.534568
299    Marqueur 89   Marqueur 90        84.882866
377   Marqueur 114  Marqueur 120        89.294875
317    Marqueur 94   Marqueur 96        90.132277
382   Marqueur 115  Marqueur 120        94.935522

Average minimum distance between points in meters: 220.62656411664315 meters
Standard deviation of the minimum distances: 54.07090348429101 meters
Standard error of the minimum distances: 2.0278173100771535 meters
The area covered by the points is: 743208563.2576514 square meters
